In [1]:
from libs.alibaba.visitor import Visitor

from libs.json import JSON
from openpyxl import load_workbook

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import os
import traceback
import types
import pendulum

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
visitor = Visitor(market, headless=False)

. storage markets.json
.\storage\markets.json was deserialized!


In [3]:
visitor.update()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-19.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-18.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-17.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-16.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-15.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-14.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-13.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-12.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-11.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-10.json was serialized!
E:\glitter\Eyelashes_config\visitors\visitors_2018-10-09.json was serialized!
E:\glitter\Eyelashes_config\

In [11]:
def crawl_current_page_visitors(self):
    css_tbody = '#J-visitors-tbl-tbody'
    tbody = WebDriverWait(self.browser, 15).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, css_tbody)))

    visitors = []
    fns = os.listdir(self.visitor_dir)

    for tr in tbody.find_elements_by_css_selector(css_tbody+' tr.J-visitors-table-tr'):
        ActionChains(self.browser).move_to_element(tr).perform()
        date = tr.find_element_by_css_selector('td.td-checkbox input').get_attribute('statdate')

        fn = 'visitors_'+date+'.json'
        if fn in fns:
            break

        visitor = {}
        visitor['id'] = tr.find_element_by_css_selector('td.td-checkbox input').get_attribute('visitorid')
        visitor['idx'] = tr.find_element_by_css_selector('td.td-checkbox input').get_attribute('visitoridx')
        visitor['date'] = date
        visitor['region'] = tr.find_element_by_css_selector('td.td-region span').get_attribute('title')

        pv_span = tr.find_element_by_css_selector('td.td-pv span')
        visitor['pv'] = pv_span.text

        visitor['pv-detail'] = []
        if visitor['pv'] != '0':
            css_pv_detail = '#J-visitor-detail'
            css_pv_detail_close = '#J-vistor-detail-close'
            css_pv_detail_body = '#J-visitor-detail-tbl-tbody'
            css_pv_detail_pagination = '#J-pagination-visitor-detail'
            pv_span.click()
            while True:
                pv_detail_tbody = WebDriverWait(self.browser, 15).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, css_pv_detail_body)))

                for pv_tr in pv_detail_tbody.find_elements_by_css_selector('tr'):
                    pv = {}
                    pv['idx'] = pv_tr.find_element_by_css_selector('td.visitor-detail-index').text
                    pv['page'] = pv_tr.find_element_by_css_selector('td.visitor-detail-page a').get_attribute('href')
                    page_acts = pv_tr.find_element_by_css_selector('td.visitor-detail-page div').text
                    if '已发起询盘' in page_acts:
                        pv['inquiried'] = True
                    else:
                        pv['inquiried'] = False

                    if '已发起TradeManager咨询' in page_acts:
                        pv['tm_inquiried'] = True
                    else:
                        pv['tm_inquiried'] = False
                    pv['stay'] = pv_tr.find_element_by_css_selector('td.visitor-detail-stay').text
                    pv['time'] = pv_tr.find_element_by_css_selector('td.visitor-detail-time').text
                    visitor['pv-detail'].append(pv)

                next_pv_detail_button = self.browser.find_elements_by_css_selector(css_pv_detail_pagination + ' .ui-pagination-active + a')
                if next_pv_detail_button:
                    while True:
                        try:
                            ActionChains(self.browser).move_to_element(next_pv_detail_button[0]).perform()
                            next_pv_detail_button[0].click()
                            break
                        except WebDriverException as e:
                            if 'is not clickable at point' in str(e):
                                self.browser.implicitly_wait(0.5)
                                continue
                            else:
                                raise e
                else:
                    break

            while True:
                try:
                    self.browser.find_element_by_css_selector(css_pv_detail_close).click()
                    break
                except WebDriverException as e:
                    if 'is not clickable at point' in str(e):
                        self.browser.implicitly_wait(0.5)
                        continue
                    else:
                        raise e

        visitor['stay'] = tr.find_element_by_css_selector('td.td-stay-duration').text

        kws_div = tr.find_elements_by_css_selector('td.td-search-keywords>div.search-keywords')
        if kws_div:
            kws_div = kws_div[0]
            kws_text = kws_div.get_attribute('data-text')
            visitor['keywords'] =  re.sub('</div><div>', ',', kws_text)[5:-6].split(',')
            visitor['search_keyword_indices'] = []
            for idx, div in enumerate(kws_div.find_elements_by_css_selector('div')):
                if div.find_elements_by_css_selector('span.search-keyword'):
                    visitor['search_keyword_indices'].append(idx)
        else:
            visitor['keywords'] = []
            visitor['search_keyword_indices'] = []

        visitor['minisite-acts'] = []
        for div in tr.find_elements_by_css_selector('td.td-minisite-active span'):
            visitor['minisite-acts'].append(div.get_attribute('textContent'))
        visitor['website-acts'] = []
        for div in tr.find_elements_by_css_selector('td.td-website-active span'):
            visitor['website-acts'].append(div.get_attribute('textContent'))

        visitors.append(visitor)

    return visitors
visitor.crawl_current_page_visitors = types.MethodType(crawl_current_page_visitors, visitor)

In [4]:
visitors = visitor.deserialize()

E:\glitter\Eyelashes_config visitors visitors_2018-08-06.json
E:\glitter\Eyelashes_config\visitors\visitors_2018-08-06.json was deserialized!
E:\glitter\Eyelashes_config visitors visitors_2018-08-07.json
E:\glitter\Eyelashes_config\visitors\visitors_2018-08-07.json was deserialized!
E:\glitter\Eyelashes_config visitors visitors_2018-08-08.json
E:\glitter\Eyelashes_config\visitors\visitors_2018-08-08.json was deserialized!
E:\glitter\Eyelashes_config visitors visitors_2018-08-09.json
E:\glitter\Eyelashes_config\visitors\visitors_2018-08-09.json was deserialized!
E:\glitter\Eyelashes_config visitors visitors_2018-08-10.json
E:\glitter\Eyelashes_config\visitors\visitors_2018-08-10.json was deserialized!
E:\glitter\Eyelashes_config visitors visitors_2018-08-11.json
E:\glitter\Eyelashes_config\visitors\visitors_2018-08-11.json was deserialized!
E:\glitter\Eyelashes_config visitors visitors_2018-08-12.json
E:\glitter\Eyelashes_config\visitors\visitors_2018-08-12.json was deserialized!
E:\gli

In [5]:
len(visitors)

1207

In [9]:
kws = {}
words = {}
expected_kws = ['volume', 'cluster', 'extension', 'flare', 'lash']
# expected_kws = ['magnet']
for v in visitors:
    if v['region'].lower() == 'china':
        continue
        
    for kw in v['keywords']:
        kw = kw.lower()
        kw = re.sub('eyelashes', 'eyelash', kw)
        kw = re.sub('lashes', 'lash', kw)
        kw = re.sub('extensions', 'extension', kw)
        
        for w in kw.split(' '):
            if w not in words:
                words[w] = 0
            words[w] = words[w] + 1
            
        for ekw in expected_kws:
            if ekw in kw:
                if kw not in kws:
                    kws[kw] = 0
                kws[kw] = kws[kw] + 1
                break

word_list = []
for w in words:
    word_list.append([w, words[w]])
sorted_words = sorted(word_list, key=lambda k: k[1], reverse=True)

kw_list = []
for kw in kws:
    kw_list.append([kw, kws[kw]])
    
sorted_kws = sorted(kw_list, key=lambda k: k[1], reverse=True)
for kw in sorted_kws:
    if 'professional' in kw[0]:
        print(kw[1], kw[0])

4 eyelash extension professional
1 lash extension professional


In [25]:
%store sorted_kws

Stored 'sorted_kws' (list)


In [15]:
import re

In [22]:
href = "//hz-productposting.alibaba.com/product/editing.htm?id=60762374394"
ali_id = re.search('\?id=(\d+)', href).group(1)

In [27]:
from openpyxl import load_workbook

In [28]:
file = 'products ranking overview.xlsx'
wb = load_workbook(file)
sheet = wb.worksheets[0]
idx = 0
for kw in sorted_kws:
    idx += 1
    cell = sheet.cell(row=idx,column=1)
    cell.value = kw[1]
    cell = sheet.cell(row=idx,column=2)
    cell.value = kw[0]
wb.save(file)